In [1]:
import pandas as pd
import numpy as np
import json
import gc

In [2]:
# DASENet data
dasenet_path = './Bug-Related-Activity-Logs/chromium.csv'

# DeepTriage data
triage_path = '../../../Data/DeepTriage/Chromium/all_data.json'

In [3]:
dasenet_df = pd.read_csv(dasenet_path)

In [4]:
dasenet_df.head()

,Unnamed: 0,bugID,user_comments,system_records,s1,s2,s3,s4,s5,s6,s7,s8,s9,y
0,0,350007,"['inactive', 'v1', 'app', 'window', 'seems', '...",[],1,1,0,24,1,0,"[0, 0, 0]",1,"[1, 0]",24
1,1,350007,"['comment1', 'kuscher', 'agree', 'looks', 'wei...","['cc', 'jennschen', 'pkotw', 'labels', 'pri-2'...",4,3,1,71,2,2,"[0, 1, 0]",3,"[0, 1]",22
2,2,350007,"['comment4', 'sgabr', 'need', 'transparency', ...",[],1,1,0,31,3,3,"[1, 0, 1]",1,"[0, 1]",21
3,3,350007,"['comment5', 'sgabr', 'marking', 'one', 'wo', ...","['status', 'wontfix']",2,1,1,14,4,7,"[0, 0, 0]",1,"[0, 1]",17
4,4,350007,"['comment6', 'pkotw', 'issue', 'has', 'merged'...","['cc', 'smokana', 'status', 'assigned']",5,3,2,31,5,8,"[0, 0, 1]",2,"[0, 1]",16


In [5]:
dasenet_ids = dasenet_df['bugID']

In [6]:
with open(triage_path, 'r', encoding='utf-8') as f:
    triage_df = json.load(f, strict=False)

In [7]:
triage_df[0]

{'id': 1,
 'issue_id': 2,
 'issue_title': 'Testing if chromium id works',
 'reported_time': '2008-08-30 16:00:21',
 'owner': '',
 'description': '\nWhat steps will reproduce the problem?\n1.\n2.\n3.\n\r\nWhat is the expected output? What do you see instead?\n\r\n\r\nPlease use labels and text to provide additional information.\n \n ',
 'status': 'Invalid',
 'type': 'Bug'}

In [8]:
triage_ids = list(map(lambda x: x['issue_id'], triage_df))

In [9]:
triage_ids[:5]

[2, 3, 4, 5, 6]

In [10]:
dasenet_ids = set(dasenet_ids)

In [11]:
triage_ids = set(triage_ids)

In [12]:
len(dasenet_ids), len(triage_ids)

(15170, 383025)

In [13]:
len(dasenet_ids.intersection(triage_ids))

14721

In [17]:
common_idx = dasenet_ids.intersection(triage_ids)

In [14]:
trimmed_df = dasenet_df.drop_duplicates(subset='bugID', keep='first')

In [15]:
len(trimmed_df)

15170

In [18]:
# Save the resulting files
len(trimmed_df[trimmed_df['bugID'].isin(common_idx)])

14721

In [19]:
trimmed_df[trimmed_df['bugID'].isin(common_idx)].to_csv('chromium-common.csv')

In [18]:
triage_common = list(filter(lambda p: p['issue_id'] in common_idx, triage_df))

In [61]:
f = open('chromium_triage_common.json', 'w')
json.dump(triage_common, f)
f.close()

In [19]:
# Add the labels to the JSON data
result = list(map(lambda p: {**p, 'y': int(np.max(dasenet_df[dasenet_df['bugID'] == p['issue_id']]['y']))}, triage_common))

In [20]:
f = open('chromium.json', 'w')
json.dump(result, f)
f.close()

In [2]:
f = open('chromium.json', 'r')
data = json.load(f)
f.close()

In [3]:
data[0]

{'id': 236,
 'issue_id': 350007,
 'issue_title': 'semitransparency of inactive v1 app window seems misleading when the window is maximized',
 'reported_time': '2014-03-06 19:00:33',
 'owner': 'sgabr...@chromium.org',
 'description': '\nGoogle Chrome\t35.0.1870.0 (Official Build 254603) dev\r\nPlatform\t5579.1.0 (Official Build) dev-channel stumpy\r\n\r\nWhat steps will reproduce the problem?\n1. open SecureShell as a window, and maximize it\r\n2. open multiple browser windows and other windows\r\n3. focus on another window\r\n4. see top right of the screen\r\n\r\nWhat is the expected output? What do you see instead?\nexpected:\r\ntitle bar is monotone-ish\r\n\r\nwhat I see:\r\nlooks like a misplaced-wrong-focus indicator-ish thing. It turns out the wallpaper with the semi-transparency of the inactive window. See the attached image.\r\nsgabriel, wdyt?  I might think better to handle differently for maximized window...\r\nIf you think this is expected, please mark as wontfix.\r\n\r\nPlea